## Data API STAC Catalog

In [1]:
import rasterio
import pystac
import boto3
import pandas as pd
import json
import datetime

from shapely.ops import unary_union
from shapely.geometry import shape

from pystac.extensions.raster import RasterExtension, RasterBand
from pystac.extensions.projection import ProjectionExtension

from stac_validator import stac_validator


This notebook creates a static STAC catalog for a Data API dataset version whose details are specified in next cell.

Credentials to S3 bucket should be available to the notebook (e.g., via AWS_PROFILE env variable) to read dataset's tiles.geojson 

In [2]:
BUCKET = 'gfw-data-lake-dev'
dataset = 'umd_glad_sentinel2_alerts'
version = 'v20220104'
epsg = '4326'
start_datetime = datetime.datetime(2000, 1, 1)
item_datetime = datetime.datetime(2022, 1, 17)
tiles_base = f'{dataset}/{version}/raster/epsg-4326/10/100000/date_conf/gdal-geotiff'
tiles_key = f'{tiles_base}/tiles.geojson'

In [3]:

stac_extensions = [
     'https://stac-extensions.github.io/raster/v1.0.0/schema.json',
     'https://stac-extensions.github.io/projection/v1.0.0/schema.json'
]

### Read tiles.geojson for a tile set which has the metadata to build a STAC asset for a dataset version

In [4]:
s3 = boto3.client('s3')

resp = s3.get_object(Key=tiles_key, Bucket=BUCKET)
geojson = json.load(resp['Body'])

df = pd.DataFrame(geojson['features'])

### Create the root STAC Catalog

In [5]:
catalog = pystac.Catalog(
    id='gfw',
    description='WRI Global Forest Watch STAC catalog.',
    href='./stac_example/catalog.json',
    stac_extensions=stac_extensions,
    catalog_type=pystac.CatalogType.SELF_CONTAINED
)

Create a standalone [collection](https://github.com/radiantearth/stac-spec/blob/4ecbabccefc66e6eb17858ae1492dffca66ab4bd/collection-spec/collection-spec.md#standalone-collections) for a dataset version with all the tiles as assets.
Standalone collections are a good way of surfacing metadata
that are the same across assets (projection, bands, time range, etc.).

In [6]:
from shapely.ops import unary_union
from shapely.geometry import shape

def get_spatial_extent(items):
    polygons = [shape(item.geometry) for item in items]
    # Returns a union of the two geojson polygons for each item
    unioned_geometry = unary_union(polygons)
    
    # Set the bbox to be the bounds of the unified polygon and return the spatial extent of the collection
    return pystac.SpatialExtent(bboxes=[unioned_geometry.bounds])



### Create STAC item for each tile set

In [7]:
items = []

for _, tile in df.iterrows():
    tile_id = tile.properties['name'].split('/')[-1].split('.')[0]
    print(tile_id)
    
    item = pystac.Item(
        id=tile_id,
        geometry=tile.geometry,
        bbox=tile.properties['extent'],
        datetime=item_datetime,
        href=f'./stac_example/{version}/{tile_id}-item.json',
        properties={}
    )
    
    item.stac_extensions = stac_extensions
    
    projection = ProjectionExtension.ext(item)
    projection.epsg = '4326'
    projection.shape = [ tile.properties['height'], tile.properties['width'] ]    # spec specifies shape in Y, X order
    
    asset_url = f'https://{BUCKET}.s3.amazonaws.com/{tiles_base}/{tile_id}.tif'
    asset = pystac.Asset(
        href=asset_url,
        title=tile_id,
        roles=["data"],
        media_type=pystac.MediaType.COG
    )
    raster = RasterExtension.ext(asset)
    
    raster.bands = [
        RasterBand(
            {
                'data_type': band['data_type'],
                'nodata': band['no_data'],
                'spatial_resolution': tile.properties['pixelxsize'],
                'statistics': {
                    'minimum': (band.get('stats', {}).get('min') if isinstance(band['stats'], dict) else None),
                    'maximum': band.get('stats', {}).get('max') if isinstance(band['stats'], dict) else None,
                    'stddev': band.get('stats', {}).get('std_dev') if isinstance(band['stats'], dict) else None
                }
            }
        ) for band in tile.properties['bands']

    ]

    item.add_asset(key=tile_id, asset=asset)
    items.append(item)    
    
    

10N_050W
10S_050W
00N_040W
00N_090W
20N_070W
10S_040W
20N_080W
10N_090W
00N_050W
10S_080W
10N_060W
10N_080W
00N_080W
10N_070W
00N_070W
00N_060W
10S_070W
10S_060W


#### Create STAC collection for a dataset version and add it to the catalog with its items

In [8]:
version_collection = pystac.Collection(
    id=version,
    title='GLAD',
    description="GLAD deforestation alert",
#     href='./stac_example/collection.json',
    extent=pystac.collection.Extent(spatial=get_spatial_extent(items), temporal=pystac.collection.TemporalExtent(intervals=[[start_datetime, item_datetime]]))
)

version_collection.add_items(items)

catalog.add_child(version_collection)

Validate the catalog and its descendants

In [9]:
stac = stac_validator.StacValidate(catalog.get_self_href(), recursive=-1)
stac.run()
validation_check = [message for message in stac.message]
validation_check

[{'version': '1.0.0',
  'path': '/Users/solomon.negusse/wri/data-api-examples/stac_example/catalog.json',
  'schema': ['https://schemas.stacspec.org/v1.0.0/catalog-spec/json-schema/catalog.json'],
  'valid_stac': True,
  'asset_type': 'CATALOG',
  'validation_method': 'recursive'},
 {'version': '1.0.0',
  'path': '/Users/solomon.negusse/wri/data-api-examples/stac_example/./v20220127/collection.json',
  'schema': ['https://schemas.stacspec.org/v1.0.0/collection-spec/json-schema/collection.json'],
  'valid_stac': True,
  'asset_type': 'COLLECTION',
  'validation_method': 'recursive'},
 {'version': '1.0.0',
  'path': '/Users/solomon.negusse/wri/data-api-examples/stac_example/./v20220127/./10N_050W-item.json',
  'schema': ['https://stac-extensions.github.io/projection/v1.0.0/schema.json',
   'https://schemas.stacspec.org/v1.0.0/item-spec/json-schema/item.json'],
  'valid_stac': True,
  'asset_type': 'ITEM',
  'validation_method': 'recursive'},
 {'version': '1.0.0',
  'path': '/Users/solomo

#### Save catalog and its children to disk

In [10]:
version_collection.save_object(include_self_link=False)

# item.save_object(include_self_link=False, dest_href=f'./stac_example/{version}/{tile_id}.json')
for item in items:
    item.save_object(include_self_link=False)

catalog.save_object()

#### Read the catalog back from disk and make sure items and collection look correct

In [11]:
cat = pystac.catalog.Catalog.from_file('./stac_example/catalog.json')

In [15]:
item = next(cat.get_all_items())
collection = next(cat.get_all_collections())
print(collection.title)

GLAD


In [13]:
# print(item.to_dict())
# print(collection.to_dict())